## Import libraries

In [1]:
import glob
import uqer
import time
import joblib
import pickle
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from uqer import DataAPI   #优矿api
from sklearn import linear_model
from influxdb import InfluxDBClient
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from pandas.core.common import SettingWithCopyWarning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

client = uqer.Client(token='18266a7c0ac9f8cdbe00f9b2ecb65f42316a5f78d9cc22ebabcbd923593356e4')
pd.options.mode.chained_assignment = None

127807@wmcloud.com 账号登录成功


In [2]:
file_location = '/data/liufengyuan/'
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open(file_location + "pct1_cal/每日预测/f_industry", "rb"))
f_x_221 = pickle.load(open(file_location + 'pct1_cal/每日预测/f_x_221', 'rb'))
label_list = ['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'openclose_pct1', 
'openclose_pct1_full', 'openclose_pct1_full_rank', 
'vwap_pct1_full', 'vwap_pct1_full_rank', 'askbid_pct1_rank', 'askbid_pct1']
f_y_list =  label_list[5]
extra_factors = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
f_x = f_x_221.copy()
f_x.extend(extra_factors)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
len(f_x)

231

## Date

In [3]:
prediction_date = datetime.date.today()
if prediction_date.isoweekday() == 1:
    prediction_date -= datetime.timedelta(days= 3)
elif prediction_date.isoweekday() in set((2, 3, 4, 5)):
    prediction_date -= datetime.timedelta(days= 1)
else:
    prediction_date -= datetime.timedelta(days= prediction_date.isoweekday() % 5)

date = prediction_date.strftime("%Y-%m-%d")
store_date = prediction_date.strftime("%Y%m%d")
select_date = (prediction_date - datetime.timedelta(days= 1)).strftime("%Y-%m-%d")
print('Prediciton factors: ', date)
print('Store location: ', store_date)
print('Loading data date: ', select_date)

Prediciton factors:  2022-08-09
Store location:  20220809
Loading data date:  2022-08-08


In [12]:
def check_missing_values(origin, new):
    print('Old length: ', len(origin))
    count1 = 0
    for file in origin:
        if file not in new:
            print(file)
            count1 += 1
    print('缺失值', count1)

    print('New length', len(new))
    count2 = 0
    for file in new:
        if file not in origin:
            print(file)
            count2 += 1
    print('多余值', count2)

    return (count1 == 0 and count2 == 0)

Useful Functions

In [185]:
start = time.time()

In [186]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## exclude extreme values
class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

In [187]:
# %% a list of classes

## fill empty values
class FillEmpty(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_mean_industry = None

    def fit(self, df_name, f_x, group_name='industry'):
        self.df_mean_industry = df_name.groupby(group_name).mean()[f_x]
        self.df_mean_industry = self.df_mean_industry.fillna(self.df_mean_industry.mean())
        self.df_mean_industry.columns = [x + '_mean' for x in self.df_mean_industry.columns]
        self.df_mean_industry = self.df_mean_industry.reset_index()
        self.df_mean_industry = self.df_mean_industry.fillna(0)  # 可以删除
        return self

    def transform(self, df_name, f_x, group_name='industry'):
        df_name_mean = df_name.merge(self.df_mean_industry, on=group_name, how='left')
        df_name_mean[f_x] = df_name_mean[f_x].apply(lambda x: x.fillna(df_name_mean[x.name + '_mean']))
        df_name = df_name_mean[df_name.columns]
        return df_name

## neutralize industry
class Neutralization(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.models = {}

    def regr_fit(self, X, y):
        self.models[y.name] = linear_model.LinearRegression().fit(X, y)
        return

    def regr_pred(self, X, y):
        pred = self.models[y.name].predict(X)
        return y - pred

    def fit(self, df_name, f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x].apply(lambda y: self.regr_fit(X, y))
        return self

    def transform(self, df_name, f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x] = df_name[f_x].apply(lambda y: self.regr_pred(X, y))
        return df_name

Factors

In [188]:
def get_data(file_loc, f_x, date, thresh):
    result_list = []
    for name in f_x:
        #print(name)
        alpha = pd.read_csv(file_loc + name + '.csv')
        if alpha.columns.values.tolist()[0] == 'date':
            alpha = alpha.rename(columns={"date": "tradeDate"})
        if file_loc != 'alphas_all/066_':
            alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
        alpha['ticker'] = alpha['ticker'].astype(int)
        if date != '':
            alpha = alpha[alpha.tradeDate == date]
        alpha = alpha.set_index(['ticker', 'tradeDate'])
        alpha = alpha.sort_index(ascending=True)
        result_list.append(alpha)
    result = pd.concat(result_list, axis=1)
    result = result.reset_index()
    result = result.dropna(thresh=thresh)
    return result

In [189]:
factors = get_data(file_location + 'pct1_cal/每日预测/data/', f_x_221, date, 67)
factors

,ticker,tradeDate,minute_new_FACTORS_CDPP,minute_new_FACTORS_Down_beta,minute_new_FACTORS_ILLIQ,minute_new_FACTORS_IMI,minute_new_FACTORS_PVC,minute_new_FACTORS_TMA_turn,minute_new_FACTORS_TSMOM,L2_FACTORS_NTO,...,x_tech_liangjia_FACTORS_ADXR,x_tech_liangjia_FACTORS_AR_26,x_tech_liangjia_FACTORS_GLVR20,x_tech_liangjia_FACTORS_MF20,x_tech_liangjia_FACTORS_TurnOverVol,x_tech_liangjia_FACTORS_factor_model_genetic_019,x_tech_liangjia_FACTORS_factor_model_genetic_028,x_tech_liangjia_FACTORS_factor_model_genetic_basic_turn,choice_FACTORS_HQFW_SIGNAL,choice_FACTORS_NEWS_SIGNAL
0,1,2022-07-25,-0.231545,NaN,9.636517e-08,28.169014,1.202067,0.006154,-562.305844,-0.0065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2022-07-25,0.631627,NaN,1.940779e-07,45.299145,2.350698,0.008790,-158.566795,0.0056,...,0.093833,110.294118,3.287215,1.598983e+09,4.221046e+07,822.085824,-3.847577e-08,821.662163,NaN,NaN
2,4,2022-07-25,-0.075333,NaN,7.058890e-06,58.510638,3.516238,0.030868,-27.636088,0.0792,...,0.145688,115.128205,0.754059,3.389273e+07,1.511160e+06,193.805463,-2.525957e-07,194.081014,NaN,NaN
3,6,2022-07-25,0.123651,NaN,2.961835e-06,40.178571,-1.153593,0.017913,-1475.536902,-0.0766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,2022-07-25,0.524636,NaN,1.799163e-06,86.709539,NaN,0.036362,52.756858,-0.4356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,601880,2022-07-25,NaN,NaN,1.210298e-06,50.000000,-3.073616,0.003619,-422.398219,0.0295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4636,601988,2022-07-25,NaN,NaN,2.877575e-07,45.161290,-2.381004,0.000692,-38701.697288,0.0062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4637,603335,2022-07-25,NaN,NaN,1.062438e-06,57.703927,NaN,0.032953,7526.186391,-4.8364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4638,603603,2022-07-25,NaN,NaN,4.285457e-07,97.583972,NaN,0.070424,6087.588832,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
factors = factors.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)

In [191]:
jiheyingzi = pd.read_csv(file_location + 'pct1_cal/每日预测/jhjj_data/jhjj_factor_{}.csv'.format(date))
jiheyingzi= jiheyingzi.drop(columns='Unnamed: 0')
jiheyingzi = jiheyingzi.rename(columns={"dataDate": "tradeDate"})
jiheyingzi = jiheyingzi.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
jiheyingzi

,ticker,tradeDate,volume,cachgPct,thecommittee,askVolume1,bidVolume1,OCVP1,Gap,Open/vwap-1,caTr,caQrr
0,1,2022-07-25,486500,-0.005460,0.929763,100,83000.0,0.006368,-0.007782,-0.002113,0.002493,0.554416
1,2,2022-07-25,245000,0.000000,-0.688533,62000,1300.0,0.003629,-0.015748,-0.003757,0.002107,0.756874
2,4,2022-07-25,36600,0.000000,0.148515,4300,7100.0,0.009717,-0.018987,0.003453,0.027550,1.082840
3,5,2022-07-25,35900,-0.005556,0.117152,200400,46300.0,0.003180,-0.016484,0.006749,0.003373,0.895262
4,6,2022-07-25,2300,0.000000,0.640878,6000,17800.0,0.000213,-0.018779,-0.007126,0.000170,0.389831
...,...,...,...,...,...,...,...,...,...,...,...,...
4356,688779,2022-07-25,13909,0.000510,0.115088,900,2369.0,0.001256,-0.039196,-0.013830,0.000721,0.325486
4357,688800,2022-07-25,5304,0.000078,0.040193,10000,2579.0,0.002011,-0.051618,-0.012903,0.004911,0.251374
4358,688819,2022-07-25,19970,-0.002332,-0.414011,1000,1191.0,0.002179,-0.084859,-0.033198,0.002050,0.423731
4359,688981,2022-07-25,41062,0.000000,-0.209800,8544,8291.0,0.002719,-0.014377,-0.001933,0.000520,0.519503


## OutData

In [192]:
def get_target(tickers, date):
    outData = DataAPI.MktEqudGet(ticker=tickers, tradeDate = date, pandas="1")
    outData['ticker'] = outData['ticker'].astype(int)
    outData = outData.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    outData['log_marketValue'] = outData.marketValue.apply(np.log)
    return outData[['ticker', 'tradeDate', 'log_marketValue']]

In [193]:
"""
strData = ticekrToStr(factors[['ticker']])
tickers = strData.ticker.unique()
tickers = tickers.tolist()
marketValue = get_target(tickers, date)
factors = factors.merge(marketValue, on=['ticker', 'tradeDate'], how='left')
factors
"""

"\nstrData = ticekrToStr(factors[['ticker']])\ntickers = strData.ticker.unique()\ntickers = tickers.tolist()\nmarketValue = get_target(tickers, date)\nfactors = factors.merge(marketValue, on=['ticker', 'tradeDate'], how='left')\nfactors\n"

## DealWithData

In [5]:
def run_df(f_x_221, date, df):
    print('----- DATE {}------'.format(date))
    pd.options.mode.chained_assignment = None
    df2 = df[df.tradeDate == date]

    mevtransformer = ExcludeExtreme()
    mevtransformer.fit(df2[f_x_221])
    df2[f_x_221] = mevtransformer.transform(df2[f_x_221])

    df2 = df2[f_index + f_x_221]
    return df2

In [195]:
jiheyingzi = pd.read_csv(file_location + 'pct1_cal/每日预测/jhjj_data/jhjj_factor_{}.csv'.format(date))
jiheyingzi= jiheyingzi.drop(columns='Unnamed: 0')
jiheyingzi = jiheyingzi.rename(columns={"dataDate": "tradeDate"})
factors = factors.merge(jiheyingzi, on=['ticker', 'tradeDate'], how='left')
test_factors = run_df(f_x, date, factors)
test_factors

----- DATE 2022-07-25------


,ticker,tradeDate,minute_new_FACTORS_CDPP,minute_new_FACTORS_Down_beta,minute_new_FACTORS_ILLIQ,minute_new_FACTORS_IMI,minute_new_FACTORS_PVC,minute_new_FACTORS_TMA_turn,minute_new_FACTORS_TSMOM,L2_FACTORS_NTO,...,volume,cachgPct,thecommittee,askVolume1,bidVolume1,caQrr,caTr,OCVP1,Open/vwap-1,Gap
0,1,2022-07-25,-0.231545,NaN,9.636517e-08,28.169014,1.202067,0.006154,-562.305844,-0.0065,...,117100.0,-0.005460,0.929763,100.0,17100.0,0.554416,0.002493,0.006368,-0.002113,-0.007782
1,2,2022-07-25,0.631627,NaN,1.940779e-07,45.299145,2.350698,0.008790,-158.566795,0.0056,...,117100.0,0.000000,-0.688533,11930.0,1300.0,0.756874,0.002107,0.003629,-0.003757,-0.015748
2,4,2022-07-25,-0.075333,NaN,7.058890e-06,58.510638,3.516238,0.030868,-27.636088,0.0792,...,36600.0,0.000000,0.148515,4300.0,7100.0,1.082840,0.016119,0.009717,0.003453,-0.018987
3,5,2022-07-25,NaN,NaN,1.529592e-05,76.744186,NaN,0.008829,-34.742164,-0.1138,...,35900.0,-0.005556,0.117152,11930.0,17100.0,0.895262,0.003373,0.003180,0.006749,-0.016484
4,6,2022-07-25,0.123651,NaN,2.961835e-06,40.178571,-1.153593,0.017913,-1475.536902,-0.0766,...,2300.0,0.000000,0.640878,6000.0,17100.0,0.389831,0.000170,0.000213,-0.007126,-0.018779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4633,688799,2022-07-25,0.346683,NaN,1.529592e-05,52.358037,-2.590596,0.020884,NaN,0.1073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4634,688800,2022-07-25,0.642194,NaN,1.529592e-05,63.223411,-1.479016,0.042184,NaN,0.0221,...,5304.0,0.000078,0.040193,10000.0,2579.0,0.251374,0.004911,0.002011,-0.012903,-0.051618
4635,688819,2022-07-25,0.865454,NaN,3.639088e-06,51.079771,2.007598,0.029308,110.106458,-0.8144,...,19970.0,-0.002332,-0.414011,1000.0,1191.0,0.423731,0.002050,0.002179,-0.031902,-0.074491
4636,688981,2022-07-25,0.809092,NaN,6.556925e-07,24.226804,2.191111,0.008354,42.940444,-0.0184,...,41062.0,0.000000,-0.209800,8544.0,8291.0,0.519503,0.000520,0.002719,-0.001933,-0.014377


In [196]:
model = joblib.load(file_location + 'pct1_cal/每日预测/train_model_new.m')
y_pred = model.predict(test_factors[f_x])

In [197]:
result = test_factors[f_index]
result['prediction'] = y_pred
result.rename(columns = {'tradeDate': 'date'}, inplace = True)
result['date'] = result.date.apply(lambda x: str(x).replace('-', ''))
result = result.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
result.to_csv(file_location + 'pct1_cal/每日预测/prediction/{}.csv'.format(store_date), index = False)
result

,ticker,date,prediction
0,301075,20220725,0.750794
1,300375,20220725,0.748891
2,300780,20220725,0.747190
3,688601,20220725,0.745714
4,300796,20220725,0.744034
...,...,...,...
4633,600265,20220725,0.473215
4634,300816,20220725,0.472983
4635,689009,20220725,0.468270
4636,2883,20220725,0.462194


In [198]:
print(f'耗时:{time.time() - start}')

耗时:46.68786311149597


## 数据库

In [6]:
table_name = 'all_Factors'
client = InfluxDBClient(host="175.25.50.116", port=12086, username="xtech", password="xtech2022", database="factor")
result = client.query("select * from /"+ table_name +"/ where time >= '{}' ".format(select_date))
result = pd.DataFrame(list(result.get_points()))

factors = result.drop(columns= 'time')
factors = factors.rename(columns= {'code': 'ticker'})
factors['ticker'] = factors['ticker'].astype(int)
factors.insert(0, 'tradeDate', factors.pop('tradeDate'))
factors.insert(0, 'ticker', factors.pop('ticker'))
factors['tradeDate'] = factors.tradeDate.apply(lambda x: x[:-9])
factors = factors.dropna(thresh=70)
if 'ga_FACTORS_sfactor175 ' in factors.columns.to_list():
    factors = factors.rename(columns={'ga_FACTORS_sfactor175 ': 'ga_FACTORS_sfactor175'})
factors = factors[factors.tradeDate == date]
#factors.fillna(value=np.nan, inplace=True)
factors

,ticker,tradeDate,HK_new_FACTORS_Weight,HK_new_FACTORS_hkVol_change_20_rate,HK_new_FACTORS_partyPct,HK_new_FACTORS_pct_change_mean_20,L2_FACTORS_ab_turnover,any_cor_FACTORS_mom_cl,choice_FACTORS_HQFW_SIGNAL,choice_FACTORS_NEWS_SIGNAL,...,x_tech_liangjia_FACTORS_ADX,x_tech_liangjia_FACTORS_ADXR,x_tech_liangjia_FACTORS_AR_26,x_tech_liangjia_FACTORS_GLVR20,x_tech_liangjia_FACTORS_MF20,x_tech_liangjia_FACTORS_TurnOverVol,x_tech_liangjia_FACTORS_factor_model_genetic_019,x_tech_liangjia_FACTORS_factor_model_genetic_028,x_tech_liangjia_FACTORS_factor_model_genetic_basic_turn,zhaopin_FACTORS_rolling_increase_rate
0,1,2022-08-09,0.003664,-2.631808e-02,7.58,-14773082.60,4.0,-0.056983,1274700.16,15069.978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.347041
1,689009,2022-08-09,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,688981,2022-08-09,-0.001258,9.549993e-04,0.30,81488.40,6.0,NaN,1622241.19,9269.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,688819,2022-08-09,NaN,NaN,NaN,NaN,2.0,NaN,242071.15,238.986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,688800,2022-08-09,NaN,NaN,NaN,NaN,5.0,NaN,33526.51,318.450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,7,2022-08-09,NaN,NaN,NaN,NaN,2.0,NaN,111037.50,143.946,...,0.186555,0.053317,63.738739,0.532822,1.127055e+08,3.802049e+06,470.121014,0.000000,466.555934,-0.818182
5356,6,2022-08-09,-0.000085,-9.862500e-04,0.21,-50465.00,4.0,-0.071490,61542.74,50.688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
5357,5,2022-08-09,-0.000031,6.368405e-08,0.00,0.00,5.0,0.000000,93876.39,463.716,...,0.155844,0.158279,162.000000,0.489265,1.504484e+07,3.522705e+06,207.706509,-0.000004,208.209462,1.125000
5358,4,2022-08-09,NaN,NaN,NaN,NaN,5.0,NaN,146881.58,1114.014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
names = pickle.load(open(file_location + 'pct1_cal/每日预测/f_x_221', 'rb'))
extra_factors = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
extra_factors = ['jhjj_FACTORS_' + x for x in extra_factors]
names.extend(extra_factors)

new_factors = factors.columns.tolist()
new_factors.remove('tradeDate')
new_factors.remove('ticker')
all_here = check_missing_values(names, new_factors)
all_here

Old length:  231
L2_FACTORS_NTO
L2_FACTORS_OI_money
L2_FACTORS_OI_volume
L2_FACTORS_Stren_21
L2_FACTORS_Stren_42
L2_FACTORS_abs_NTO
L2_FACTORS_f1
L2_FACTORS_f2
L2_FACTORS_f3
L2_FACTORS_f4
L2_FACTORS_f5
L2_FACTORS_f6
L2_FACTORS_f7
L2_FACTORS_f8
L2_FACTORS_f9
ga_FACTORS_sfactor175
缺失值 16
New length 215
多余值 0


False

In [11]:
#tickers = test_factors.ticker.unique().tolist()
try:
    factors = run_df(names, date, factors)
except TypeError:
    result = factors.dtypes
    result.drop('tradeDate', inplace = True)
    print('Empty factors: \n{}'.format(result[result == object]))
#factors = factors[factors['ticker'].isin(tickers)]

----- DATE 2022-08-09------


KeyError: "['L2_FACTORS_NTO', 'L2_FACTORS_OI_money', 'L2_FACTORS_OI_volume', 'L2_FACTORS_Stren_21', 'L2_FACTORS_Stren_42', 'L2_FACTORS_abs_NTO', 'L2_FACTORS_f1', 'L2_FACTORS_f2', 'L2_FACTORS_f3', 'L2_FACTORS_f4', 'L2_FACTORS_f5', 'L2_FACTORS_f6', 'L2_FACTORS_f7', 'L2_FACTORS_f8', 'L2_FACTORS_f9', 'ga_FACTORS_sfactor175'] not in index"

In [34]:
if result is not None:
    print('j')

j


In [45]:
empty_fraction = factors['L2_FACTORS_NTO'].isnull().sum() / factors['L2_FACTORS_NTO'].shape[0]
print('Empty Fraction is {}'.format(empty_fraction))

Empty Fraction is 1.0


In [53]:
result = factors.dtypes
result.drop('tradeDate', inplace = True)
empty_factors = pd.DataFrame(result[result == object])
empty_factors.reset_index(inplace=True)
empty_factors.rename(columns={0: 'type', 'index': 'name'}, inplace=True)
empty_factors['Empty_fraction'] = empty_factors.name.apply(lambda x: (factors[x].isnull().sum() / factors[x].shape[0]))
empty_factors.set_index('name', inplace=True)
display(empty_factors)

,type,Empty_fraction
name,,
L2_FACTORS_NTO,object,1.0
L2_FACTORS_OI_money,object,1.0
L2_FACTORS_OI_volume,object,1.0
L2_FACTORS_Stren_21,object,1.0
L2_FACTORS_Stren_42,object,1.0
L2_FACTORS_abs_NTO,object,1.0
L2_FACTORS_f1,object,1.0
L2_FACTORS_f2,object,1.0
L2_FACTORS_f3,object,1.0


In [10]:
model = joblib.load('pct1_cal/每日预测/train_model_new.m')
y_pred = model.predict(factors[names])

#整理预测结果
result = factors[f_index]
result['prediction'] = y_pred
result.rename(columns = {'tradeDate': 'date'}, inplace = True)
result['date'] = result.date.apply(lambda x: str(x).replace('-', ''))
result = result.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
#result.to_csv('pct1_cal/每日预测/prediction/{}.csv'.format(store_date), index = False)
result

,ticker,date,prediction
0,301195,20220808,0.636017
1,688375,20220808,0.634891
2,301269,20220808,0.634645
3,603170,20220808,0.632684
4,688231,20220808,0.631726
...,...,...,...
4726,688398,20220808,0.408659
4727,2938,20220808,0.407969
4728,300654,20220808,0.406509
4729,300791,20220808,0.406319


## Research

In [25]:
alpha_list = []
for num in range(0, 66):
    alpha_list.append('alpha_{}'.format(num))
alpha_data = pd.read_csv('pct1_cal/每日预测/alpha_data.csv')
dates = alpha_data.tradeDate.sort_values().unique()

In [27]:
print('PARALLEL')
start = time.time()
parallel_obj = Parallel(n_jobs=-1)(delayed(run_df)(alpha_list, date, alpha_data) for date in dates)
print(f'耗时:{time.time() - start}')
alpha_data1 = pd.concat(parallel_obj)

PARALLEL
----- DATE 2017-01-23------
----- DATE 2017-01-24------
----- DATE 2017-01-25------
----- DATE 2017-01-26------
----- DATE 2017-02-03------
----- DATE 2017-02-06------
----- DATE 2017-02-07------
----- DATE 2017-02-08------
----- DATE 2017-02-09------
----- DATE 2017-02-10------
----- DATE 2017-02-13------
----- DATE 2017-02-14------
----- DATE 2017-02-15------
----- DATE 2017-02-16------
----- DATE 2017-02-17------
----- DATE 2017-02-20------
----- DATE 2017-02-21------
----- DATE 2017-02-22------
----- DATE 2017-02-23------
----- DATE 2017-02-24------
----- DATE 2017-02-27------
----- DATE 2017-02-28------
----- DATE 2017-03-01------
----- DATE 2017-03-02------
----- DATE 2017-03-03------
----- DATE 2017-03-06------
----- DATE 2017-03-07------
----- DATE 2017-03-08------
----- DATE 2017-03-09------
----- DATE 2017-03-10------
----- DATE 2017-03-13------
----- DATE 2017-03-14------
----- DATE 2017-03-15------
----- DATE 2017-03-16------
----- DATE 2017-03-17------
----- DATE 

In [28]:
display(alpha_data)

,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2017-01-23,0.505398,0.926000,0.733400,NaN,NaN,NaN,NaN,NaN,...,0.232500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.516000
1,1,2017-01-24,0.503890,0.908000,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.234500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.540000
2,1,2017-01-25,0.505563,0.918500,0.866700,NaN,NaN,NaN,NaN,NaN,...,0.244400,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.532000
3,1,2017-01-26,0.503347,0.911000,0.933000,NaN,NaN,NaN,NaN,NaN,...,0.246200,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.533000
4,1,2017-02-03,0.503449,0.909700,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.248000,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.629000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983993,689009,2022-07-22,NaN,0.718109,NaN,0.581942,NaN,NaN,0.502263,0.500109,...,0.749190,NaN,NaN,-5.093013,0.500998,0.500996,0.933333,0.500998,15.0,NaN
4983994,689009,2022-07-25,0.532273,0.591823,0.800000,0.585736,NaN,NaN,0.502371,0.500109,...,0.752544,NaN,NaN,-14.398698,0.500998,0.500996,0.833333,0.500998,NaN,0.540234
4983995,689009,2022-07-26,0.514619,0.669616,0.866667,0.587814,NaN,NaN,0.502156,0.500109,...,0.750539,NaN,NaN,1.956526,0.500998,0.500996,0.833333,0.500998,NaN,0.515320
4983996,689009,2022-07-27,0.537030,0.745145,0.933333,0.591175,NaN,NaN,0.501940,0.500109,...,0.751079,NaN,NaN,7.059825,0.500998,0.500996,0.933333,0.500998,NaN,0.551398


In [29]:
display(alpha_data)
alpha_data1 = alpha_data1.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
display(alpha_data1)

,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2017-01-23,0.505398,0.926000,0.733400,NaN,NaN,NaN,NaN,NaN,...,0.232500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.516000
1,1,2017-01-24,0.503890,0.908000,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.234500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.540000
2,1,2017-01-25,0.505563,0.918500,0.866700,NaN,NaN,NaN,NaN,NaN,...,0.244400,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.532000
3,1,2017-01-26,0.503347,0.911000,0.933000,NaN,NaN,NaN,NaN,NaN,...,0.246200,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.533000
4,1,2017-02-03,0.503449,0.909700,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.248000,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.629000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983993,689009,2022-07-22,NaN,0.718109,NaN,0.581942,NaN,NaN,0.502263,0.500109,...,0.749190,NaN,NaN,-5.093013,0.500998,0.500996,0.933333,0.500998,15.0,NaN
4983994,689009,2022-07-25,0.532273,0.591823,0.800000,0.585736,NaN,NaN,0.502371,0.500109,...,0.752544,NaN,NaN,-14.398698,0.500998,0.500996,0.833333,0.500998,NaN,0.540234
4983995,689009,2022-07-26,0.514619,0.669616,0.866667,0.587814,NaN,NaN,0.502156,0.500109,...,0.750539,NaN,NaN,1.956526,0.500998,0.500996,0.833333,0.500998,NaN,0.515320
4983996,689009,2022-07-27,0.537030,0.745145,0.933333,0.591175,NaN,NaN,0.501940,0.500109,...,0.751079,NaN,NaN,7.059825,0.500998,0.500996,0.933333,0.500998,NaN,0.551398


,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2017-01-23,0.505398,0.926000,0.733400,NaN,NaN,NaN,NaN,NaN,...,0.732400,NaN,NaN,NaN,0.514400,0.514000,NaN,0.515000,NaN,0.516000
1,1,2017-01-24,0.503890,0.908000,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.734400,NaN,NaN,NaN,0.519000,0.514000,NaN,0.519000,NaN,0.540000
2,1,2017-01-25,0.505563,0.918500,0.866700,NaN,NaN,NaN,NaN,NaN,...,0.744000,NaN,NaN,NaN,0.521000,0.514000,NaN,0.519000,NaN,0.532000
3,1,2017-01-26,0.503347,0.911000,0.933000,NaN,NaN,NaN,NaN,NaN,...,0.746000,NaN,NaN,NaN,0.520000,0.514000,NaN,0.520000,NaN,0.533000
4,1,2017-02-03,0.503449,0.909700,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.748000,NaN,NaN,NaN,0.521000,0.514000,NaN,0.521000,NaN,0.629000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983993,689009,2022-07-22,NaN,0.718109,NaN,0.581942,NaN,NaN,0.502263,0.500109,...,0.749190,NaN,NaN,-5.093013,0.500998,0.500996,0.933333,0.500998,15.0,NaN
4983994,689009,2022-07-25,0.532273,0.591823,0.800000,0.585736,NaN,NaN,0.502371,0.500109,...,0.252544,NaN,NaN,-14.398698,0.500998,0.500996,0.833333,0.500998,NaN,0.540234
4983995,689009,2022-07-26,0.514619,0.669616,0.866667,0.587814,NaN,NaN,0.502156,0.500109,...,0.250539,NaN,NaN,1.956526,0.500998,0.500996,0.833333,0.500998,NaN,0.515320
4983996,689009,2022-07-27,0.537030,0.745145,0.933333,0.591175,NaN,NaN,0.501940,0.500109,...,0.251079,NaN,NaN,7.059825,0.500998,0.500996,0.933333,0.500998,NaN,0.551398


In [30]:
alpha_data1.to_csv('pct1_cal/每日预测/modified_alpha_data.csv', index=False)

In [ ]:
data_factors = get_data('pct1_cal/每日预测/data/', f_x_221, '')
data_factors
data_factors2 = get_data('data/', f_x_221, '')
data_factors2

In [ ]:
training_data = pd.concat([alternative, data_factors, data_factors2], axis=0)
training_data = training_data.drop_duplicates(subset=['ticker', 'tradeDate']).dropna(thresh = 67)
training_data = training_data.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
training_data = reduce_mem_usage(training_data)
training_data.to_csv('pct1_cal/每日预测/data_train.csv', index=False)

In [ ]:
dates = jingjia_data_train.tradeDate.sort_values().unique()
print('PARALLEL')
start = time.time()
parallel_obj = Parallel(n_jobs=-1)(delayed(run_df)(f_x, date, jingjia_data_train) for date in dates)
print(f'耗时:{time.time() - start}')
train_factors = pd.concat(parallel_obj)

In [ ]:
train_factors.to_csv('pct1_cal/每日预测/modified_train.csv', index=False)
train_factors = train_factors.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
train_factors

In [30]:
alpha = pd.read_csv('labels/openclose_pct1_full.csv')

In [31]:
df1 = train_factors.merge(alpha, on=['ticker', 'tradeDate'], how='left')
df1 = df1[~df1.openclose_pct1_full_rank.isnull()]
df1

,ticker,tradeDate,volume,cachgPct,thecommittee,askVolume1,bidVolume1,caQrr,caTr,OCVP1,...,x_tech_liangjia_FACTORS_GLVR20,x_tech_liangjia_FACTORS_MF20,x_tech_liangjia_FACTORS_TurnOverVol,x_tech_liangjia_FACTORS_factor_model_genetic_019,x_tech_liangjia_FACTORS_factor_model_genetic_028,x_tech_liangjia_FACTORS_factor_model_genetic_basic_turn,choice_FACTORS_HQFW_SIGNAL,choice_FACTORS_NEWS_SIGNAL,openclose_pct1_full,openclose_pct1_full_rank
0,1,2017-01-03,48800.0,-0.001092,-0.1528,14565.0,11700.0,0.825084,0.001289,0.004820,...,NaN,NaN,NaN,NaN,NaN,594.5,NaN,NaN,0.002186,0.296830
1,1,2017-01-04,71700.0,0.001092,-0.2580,16500.0,13400.0,1.400541,0.001810,0.006910,...,NaN,NaN,NaN,NaN,NaN,670.5,NaN,NaN,0.000000,0.574832
2,1,2017-01-05,63400.0,0.000000,-0.3882,16500.0,13500.0,1.653784,0.002990,0.008908,...,NaN,NaN,NaN,587.00,-1.417341e-05,587.0,NaN,NaN,-0.004362,0.564217
3,1,2017-01-06,67403.0,0.000000,-0.2440,14800.0,13500.0,0.416832,0.001246,0.005978,...,NaN,NaN,NaN,597.50,-0.000000e+00,598.5,NaN,NaN,0.002191,0.418716
4,1,2017-01-09,55300.0,0.000000,-0.1225,14100.0,11700.0,0.497564,0.000620,0.002949,...,NaN,NaN,NaN,600.50,-3.207217e-06,601.0,NaN,NaN,-0.001093,0.589793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353581,689009,2022-04-20,501.0,-0.007000,-0.8450,4334.0,200.0,1.670000,0.000070,0.000879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.035509,0.577406
4353582,689009,2022-04-21,200.0,-0.003801,-0.6406,407.0,200.0,0.399202,0.000028,0.000285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006768,0.660326
4353583,689009,2022-04-25,5213.0,0.022860,-0.4160,606.0,500.0,2.498824,0.000753,0.005226,...,0.009400,30875384.0,228628.75,98.30,1.043650e-08,99.9,NaN,NaN,-0.021788,0.764313
4353584,689009,2022-04-26,15900.0,0.012430,-0.4550,519.0,225.0,3.050067,0.002274,0.018170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.015420,0.073161


In [32]:
#e: extra, m: filted
data_source = 'full_Alter_extra_221'
file_location = 'pct1_cal'
file_type = 'full'
model_list = ['LinearRegression', 'RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = model_list[4]
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[0]

In [33]:
dates = df1.tradeDate.sort_values().unique()
epoch_ts = list(dates)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31' '2019-02-01'
test_fi = len(epoch_ts) - 1 # excluded. '2019-01-16'

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / update)
epoch_range = range(0, num_epoch + 1)
timediff = pd.Timedelta(100,unit='d')

In [34]:
def load_lrlist(lr_max,lr_min,num_rounds):
    lrlist = [lr_max+(lr_min-lr_max)*(np.log(i)/np.log(num_rounds)) for i in range(1,num_rounds+1)]
    return lrlist

lr_max = 0.05
lr_min = 0.02
n_jobs = -1

#gbm params
params =  {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'lambda_l1':1,
    'lambda_l2':10,
    'max_bin' : 64,
    'num_boost_round': 200,
    'learning_rate' : 0.04,
    'min_data_in_leaf':10,
    'num_threads': n_jobs
}
params['objective'] = 'regression'
params['metric'] = {'l2', 'auc'}
lrlist = load_lrlist(lr_max,lr_min,params['num_boost_round'])
params['lrlist'] = lrlist

In [42]:
f_y = f_y_list
result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
print(result_name)

df_result_all = pd.DataFrame()
total_IC = 0
start = time.time()
#for epoch in epoch_range:
epoch = num_epoch
print('----- EPOCH {}------'.format(epoch))
update_n = epoch * update
# get a list of train dates
epoch_t_train = epoch_ts[train_si + update_n :]
# get a list of test dates
df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
#print('训练时间：', epoch_t_train)
# 获得 x
x_train = df_train[f_x].values

# 获得y
y_train = df_train[f_y].values
lgb_train = lgb.Dataset(x_train, y_train)
model = lgb.train(params,
                lgb_train,
                num_boost_round=params['num_boost_round'],
                callbacks = [lgb.reset_parameter(learning_rate = params['lrlist'])]
                )

full_Alter_extra_221_LGBMRegressor_openclose_pct1_full_rank_r
----- EPOCH 36------
训练时间： ['2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-08', '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-29', '2020-06-30', '2020-07-01', '2020-07-02', '2020-07-03', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10', '2020-07-13', 

/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.705098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14651
[LightGBM] [Info] Number of data points in the train set: 1961396, number of used features: 231
[LightGBM] [Info] Start training from score 0.499658


In [43]:
joblib.dump(model, "pct1_cal/每日预测/train_model_new2.m")

['pct1_cal/每日预测/train_model_new.m']

In [28]:
model = joblib.load('pct1_cal/每日预测/train_model.m')

In [31]:
y_pred = model.predict(test_factors[f_x])

In [32]:
result = test_factors[f_index]
result['prediction'] = y_pred
result.rename(columns = {'tradeDate': 'date'}, inplace = True)
result = result.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
result.to_csv('pct1_cal/每日预测/prediction/{}.csv'.format(store_date), index = False)
result

,ticker,date,prediction
0,2483,2022-07-25,0.592108
1,688255,2022-07-25,0.591349
2,300672,2022-07-25,0.587936
3,603220,2022-07-25,0.585714
4,300626,2022-07-25,0.585639
...,...,...,...
4633,2799,2022-07-25,0.346860
4634,678,2022-07-25,0.344321
4635,2633,2022-07-25,0.340549
4636,603335,2022-07-25,0.335778


In [24]:
print(f'耗时:{time.time() - start}')

耗时:95.97676277160645
